# **Exract Label**

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 48.9 MB/s eta 0:00:00


In [ ]:
import rasterio
import geopandas as gpd
from rasterio.features import rasterize
import numpy as np
import os
from google.colab import drive

# Paths to the input files
drive.mount('/content/gdrive')
raster_path = "## YOUR FOLDER ##"
shapefile_path = "## YOUR FOLDER ##"
output_mask_path = "## YOUR FOLDER ##"

# Step 1: Load the raster and shapefile
with rasterio.open(raster_path) as src:
    raster_meta = src.meta
    transform = src.transform
    raster_crs = src.crs
    image_shape = src.shape

shapes = gpd.read_file(shapefile_path)

# Step 2: Reproject the shapefile if needed
if shapes.crs != raster_crs:
    shapes = shapes.to_crs(raster_crs)

# Step 3: Create a binary mask
# Convert shapes to raster coordinates
geometry = shapes.geometry
mask = rasterize(
    [(geom, 1) for geom in geometry],
    out_shape=image_shape,
    transform=transform,
    fill=0,  # Assign 0 to areas outside the shape
    dtype=np.uint8
)

# Step 4: Save the mask as a new raster file
raster_meta.update(
    dtype=rasterio.uint8,
    count=1,
    compress="lzw"
)

with rasterio.open(output_mask_path, "w", **raster_meta) as dst:
    dst.write(mask, 1)

print(f"Binary mask saved to {output_mask_path}")